In [1]:
from shap_util import *

2022-07-27 00:14:05.143390: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 00:14:05.143416: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [52]:
class Model():
    def predict(self, x):
        num_step = x.shape[1]
        num_feature = x.shape[2]
        idx = np.arange(num_step)
        idx = idx[..., np.newaxis]
        x = x*idx
        return np.sum(x)


model = Model()
relevence = np.array([[[6,10,8,9,7,5,4,3,2,1],
                              [20,19,18,17,16,15,14,13,12,11]]])/20
input_x = np.array([[[1,2,3,4,5,6,7,8,9,10],
                              [11,12,13,14,15,16,17,18,19,20]]])
relevence = relevence.transpose((0,2,1))
input_x = input_x.transpose((0,2,1))

model_type='lstm'
percentile = 75
eval_type = 'sqnc_eval'
seq_len = 3
input_new = deepcopy(input_x)
relevence = np.absolute(relevence)

# TO DO: Add other type of models
if model_type == 'grud':
    input_ts = input_x[0]
    input_new_ts = input_new[0]
elif model_type == 'lstm':
    input_ts = input_x
    input_new_ts = input_new

assert len(input_ts.shape)==3 # the time sereis data needs to be 3-dimensional

top_steps = int((1 - percentile/100) * input_ts.shape[1])
top_indices = np.argsort(relevence, axis=1)[:, -top_steps:, :]


In [54]:
# Evaluating different metrics
for p in range(input_ts.shape[0]):
    for v in range(input_ts.shape[2]):
        for t in top_indices[p,:,v]:
            if eval_type == 'prtb':
                input_new_ts[p,t,v] = np.max(input_ts[p,:,v]) - input_ts[p,t,v]
            elif eval_type == 'sqnc_eval':
                input_new_ts[p, t:t + seq_len, v] = np.mean(input_ts[p, t:t + seq_len, v])
model.predict(input_new)

1098

In [53]:
top_indices

array([[[3, 1],
        [1, 0]]])

In [50]:
top_steps

2

In [55]:
input_new_ts

array([[[ 1, 12],
        [ 3, 12],
        [ 3, 12],
        [ 3, 13],
        [ 5, 15],
        [ 5, 16],
        [ 7, 17],
        [ 8, 18],
        [ 9, 19],
        [10, 20]]])